<a href="https://colab.research.google.com/github/freddierice/semgrep-finetune/blob/main/semgrep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First make sure
1) You are using a T4 GPU instance and
2) You pip install all the libraries!

In [1]:
!pip install transformers datasets evaluate accelerate bitsandbytes peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  

Import the required libraries

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, DataCollatorForLanguageModeling, Trainer, TrainingArguments, HfArgumentParser
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from peft import PeftModel
from datasets import load_dataset
from datasets import Dataset
import numpy as np
import os

I.. I messed up ok, but this is how we are going to tokenize the text. The newlines were added because I noticed that the Llama instruction tuning did something similar. Unfortunately, I think it caused the model to learn newlines way too much.

In [3]:
def test_to_tokens(test, tokenizer):
    # processed = "<|SOC|>%s</s><|SOT|>" % (test, )
    processed = "<|SOC|>\n\n%s\n\n</s><|SOT|>\n\n" % (test, )
    return tokenizer(processed, return_tensors="pt")

Here we can load the model. It may take ~ 15 minutes to download, but it will get there. The base model is pretty big, but the LoRA weights aren't that bad.

In [4]:

model_id = 'codellama/CodeLlama-7b-hf'
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained('freddierice/semgrep-weak-7b')
tokenizer.pad_token = tokenizer.eos_token
llama_model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")
llama_model.resize_token_embeddings(len(tokenizer))
model = PeftModel.from_pretrained(llama_model, 'freddierice/semgrep-weak-7b', device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.44k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/602 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/671 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/571M [00:00<?, ?B/s]

Let's try it out! Put in some test code, add your "ruleid" and "ok" comments, and let it rip!

In [6]:
TEST="""
# cf. https://github.com/hadolint/hadolint/wiki/DL4003

FROM busybox
# ruleid: multiple-cmd-instructions
CMD /bin/true
CMD /bin/false
"""

tokens = test_to_tokens(TEST, tokenizer)
input_ids = tokens['input_ids'].to('cuda')
attention_mask = tokens['attention_mask'].to('cuda')
output = model.generate(input_ids, attention_mask=attention_mask, max_new_tokens=512)
filling = tokenizer.batch_decode(output[:, input_ids.shape[1]:], skip_special_tokens = True)[0]
print(filling.strip())

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


rules:
- id: multiple-cmd-instructions
  languages:
  - generic
  message: Multiple CMD instructions. The last CMD instruction will be used when
    running the container.
  pattern-either:
  - pattern: |
      CMD ...
      ...
      CMD ...
  - pattern: |
      CMD ...
      ...
      CMD
  severity: ERROR
